<a href="https://colab.research.google.com/github/Greeshu-09/AISECT-AIML/blob/main/Cross_valid_DTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [9]:


param_grid = param_grid = {
    'criterion': ['gini', 'entropy'],   # skip log_loss for speed
    'max_depth': [None, 5, 10],         # fewer depth options
    'min_samples_split': [2, 5],        # just 2 values
    'min_samples_leaf': [1, 2],         # keep it small
    'class_weight': [None, 'balanced']  # handle imbalance if needed
}

# GridSearchCV setup
grid = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    scoring='f1_weighted',   # classification metric
    cv=5,
    refit=True,
    verbose=3,
    n_jobs=-1
)

# Fit the model
grid.fit(X, y)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5]},
             scoring='f1_weighted', verbose=3)

In [10]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [11]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [12]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}
The f1_macro: 0.8576669285153181
The confusion Matrix:
 [[238  19]
 [ 37 106]]
The report:
               precision    recall  f1-score   support

           0       0.87      0.93      0.89       257
           1       0.85      0.74      0.79       143

    accuracy                           0.86       400
   macro avg       0.86      0.83      0.84       400
weighted avg       0.86      0.86      0.86       400



In [16]:
# Save Best Model
import pickle
filename = "DecisionTreeClassifier.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [17]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004096,0.000507,0.004958,0.000761,None,gini,None,1,2,"{'class_weight': None, 'criterion': 'gini', 'm...",0.716667,0.912121,0.843455,0.779062,0.791490,0.808559,0.065648,29
1,0.003270,0.000163,0.004805,0.000547,None,gini,None,1,5,"{'class_weight': None, 'criterion': 'gini', 'm...",0.716667,0.872507,0.829164,0.791113,0.791113,0.800113,0.051401,41
2,0.003213,0.000404,0.004505,0.000370,None,gini,None,2,2,"{'class_weight': None, 'criterion': 'gini', 'm...",0.698729,0.909009,0.855447,0.803125,0.752962,0.803854,0.073975,35
3,0.004410,0.000339,0.004789,0.000435,None,gini,None,2,5,"{'class_weight': None, 'criterion': 'gini', 'm...",0.709232,0.922545,0.869545,0.791113,0.765803,0.811648,0.075714,24
4,0.004396,0.000856,0.004919,0.000816,None,gini,5,1,2,"{'class_weight': None, 'criterion': 'gini', 'm...",0.783072,0.922545,0.897009,0.791113,0.753771,0.829502,0.067199,7


In [22]:
# Get user input
age = int(input("Enter Age: "))
salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (0 for Female, 1 for Male): "))

# Preprocess input
# The order of features should match the training data: 'Age', 'EstimatedSalary', 'Gender_Male'
user_input = np.array([[age, salary, gender_male]])
user_input_scaled = sc.transform(user_input)

# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

# Make prediction
prediction = loaded_model.predict(user_input_scaled)

# Display the prediction
if prediction[0] == 1:
    print("The user is likely to purchase.")
else:
    print("The user is unlikely to purchase.")

Enter Age: 36
Enter Estimated Salary: 33000
Enter Gender (0 for Female, 1 for Male): 1
The user is unlikely to purchase.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
